## Overview of AWS package

This is a set of tools for managing AWS for data science projects. In particular it separates the data and environment from the AWS server. This enables setup of data and programs on a free tier version before being moved to a more expensive GPU for the processing. It also enables the use of spot instances with persistent data which can save 80% of the cost of GPUs.

## Notes
    
#### How are data and programs retained?
* boot volume runs nvidia-docker
* pdrive volume is created based on most recent "cats" snapshot (or empty volume)
* pdrive is mounted as /v1
* pdrive holds docker database and program data
* on termination pdrive "cats" volume is saved to a "cats" snapshot
* if AWS initiates termination then "cats" volume needs to be saved manually
* all snapshots are retained until manually deleted

#### Why snapshots?
* cheaper storage
* can be mounted when instance created (volume cannot)
* can be attached in any availability_zone (volume is in one zone and instance would need to be in same zone)

## Setup

In [62]:
from analysis.ipstartup import *
import aws
import itools
import apps
from pdrive import Pdrive
import fabric.api as fab
from config import user

fab.env.host_string = aws.get("sm1").public_ip_address
fab.env.user = user
fab.output['everything'] = True

[root:INFO]:starting (cellevents.py\31, time=20:53)


time: 1.11 s


In [ ]:
# one off setup
1/0
aws.create_key()
aws.create_securityGroup()
aws.client.allocate_address()

## Creating and working with a new instance with a pdrive (persistent drive)

In [63]:
# create instance with 10GB volume attached called "cats". itype is free or gpu.
itools.create("gpu", bootsize=None, itype="gpu", spot=True, pdrive="cats", pdrivesize=10, docker="/v1")
apps.restart_notebook()

[root:INFO]:starting (cellevents.py\31, time=20:53)
[root:INFO]:spot request submitted (itools.py\114, time=20:53)
[root:INFO]:spot request fulfilled (itools.py\130, time=20:54)
[root:INFO]:waiting for instance running (itools.py\64, time=20:54)
[root:INFO]:instance gpu running at 34.249.163.64 (itools.py\75, time=20:54)
[root:INFO]:waiting for ssh (itools.py\148, time=20:54)
[root:INFO]:ssh connected (itools.py\157, time=20:55)


[34.249.163.64] sudo: mkdir -p /v1
[34.249.163.64] sudo: mount /dev/xvdf /v1


[root:INFO]:volume mounted (pdrive.py\101, time=20:55)


[34.249.163.64] sudo: yum install docker -y
[34.249.163.64] out: Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper
[34.249.163.64] out: 2 packages excluded due to repository priority protections
[34.249.163.64] out: Resolving Dependencies
[34.249.163.64] out: --> Running transaction check
[34.249.163.64] out: ---> Package docker.x86_64 0:1.12.6-1.17.amzn1 will be installed
[34.249.163.64] out: --> Processing Dependency: xfsprogs for package: docker-1.12.6-1.17.amzn1.x86_64
[34.249.163.64] out: --> Running transaction check
[34.249.163.64] out: ---> Package xfsprogs.x86_64 0:3.2.2-2.20.amzn1 will be installed
[34.249.163.64] out: --> Finished Dependency Resolution
[34.249.163.64] out: 
[34.249.163.64] out: Dependencies Resolved
[34.249.163.64] out: 
[34.249.163.64] out: ================================================================================
[34.249.163.64] out:  Package        Arch         Version                   Repository          Size
[34.249.163

[root:INFO]:docker installed. if need to pull images then use ssh as this shows progress whereas fabric does not (apps.py\37, time=20:56)



[34.249.163.64] sudo: chmod +x /usr/local/bin/docker-compose
[34.249.163.64] sudo: mkdir -p /etc/docker
[34.249.163.64] put: docker/daemon.json -> /etc/docker/daemon.json
[34.249.163.64] sudo: mkdir -p /v1/docker
[34.249.163.64] sudo: service docker start
[34.249.163.64] out: Starting docker:	.[  OK  ]
[34.249.163.64] out: 
[34.249.163.64] out: 

[34.249.163.64] sudo: wget -P /tmp https://github.com/NVIDIA/nvidia-docker/releases/download/v1.0.0/nvidia-docker_1.0.0_amd64.tar.xz
[34.249.163.64] out: --2017-02-18 20:56:12--  https://github.com/NVIDIA/nvidia-docker/releases/download/v1.0.0/nvidia-docker_1.0.0_amd64.tar.xz
[34.249.163.64] out: Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
[34.249.163.64] out: Connecting to github.com (github.com)|192.30.253.113|:443... connected.
[34.249.163.64] out: HTTP request sent, awaiting response... 302 Found
[34.249.163.64] out: Location: https://github-cloud.s3.amazonaws.com/releases/45557469/329876a2-dd88-11e6-8eaa-692a2a93c

ec2.Instance(id='i-0d7eac649f26ae883')

time: 2min 17s


In [67]:
apps.restart_notebook()

[root:INFO]:starting (cellevents.py\31, time=21:07)


[34.249.163.64] run: docker rm -f notebook || true
[34.249.163.64] out: Error response from daemon: No such container: notebook
[34.249.163.64] out: 

[34.249.163.64] sudo: nvidia-smi
[34.249.163.64] out: Sat Feb 18 21:07:44 2017       
[34.249.163.64] out: +------------------------------------------------------+                       
[34.249.163.64] out: | NVIDIA-SMI 352.99     Driver Version: 352.99         |                       
[34.249.163.64] out: |-------------------------------+----------------------+----------------------+
[34.249.163.64] out: | GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
[34.249.163.64] out: | Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
[34.249.163.64] out: |===============================+======================+======================|
[34.249.163.64] out: |   0  Tesla K80           On   | 0000:00:1E.0     Off |                    0 |
[34.249.163.64] out: | N/A   26C    P8    30W / 149W |  

In [68]:
itools.terminate("gpu", False)

[root:INFO]:starting (cellevents.py\31, time=21:17)
[root:INFO]:docker stopped (apps.py\73, time=21:17)
[root:INFO]:instance terminated (itools.py\178, time=21:17)
[root:INFO]:waiting until volume available (pdrive.py\135, time=21:17)
[root:INFO]:volume available (pdrive.py\137, time=21:18)
[root:INFO]:volume deleted (pdrive.py\142, time=21:19)


time: 1min 37s


## Working with an existing pdrive

In [ ]:
pdrive = Pdrive("cats")
pdrive.connect("sm1")

In [ ]:
pdrive.disconnect()

## Utilities

In [ ]:
# get a resource by name
aws.get("sm1")

In [ ]:
# get all resources (instances, volumes, snapshots)
aws.get(unique=False)

In [ ]:
# show instances used
aws.get_instances()

In [ ]:
# show python tasks running in containers
itools.get_tasks("python")

In [ ]:
# show all tasks running in containers
itools.get_tasks()

## Change docker location 

In [ ]:
# set to pdrive
apps.set_docker_folder("/v1")

In [ ]:
# set to boot drive
apps.set_docker_folder()